<a href="https://colab.research.google.com/github/guillermoayllon/certificate_tensorflow/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import model_selection
import os

In [3]:
def connect_to_tpu(tpu_address: str = None):
    if tpu_address is not None:  # When using GCP
        cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
            tpu=tpu_address)
        if tpu_address not in ("", "local"):
            tf.config.experimental_connect_to_cluster(cluster_resolver)
        tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
        strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
        print("Running on TPU ", cluster_resolver.master())
        print("REPLICAS: ", strategy.num_replicas_in_sync)
        return cluster_resolver, strategy
    else:                           # When using Colab or Kaggle
        try:
            cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
            strategy = tf.distribute.experimental.TPUStrategy(cluster_resolver)
            print("Running on TPU ", cluster_resolver.master())
            print("REPLICAS: ", strategy.num_replicas_in_sync)
            return cluster_resolver, strategy
        except:
            print("WARNING: No TPU detected.")
            mirrored_strategy = tf.distribute.MirroredStrategy()
            return None, mirrored_strategy

cluster_resolver, strategy = connect_to_tpu()

Running on TPU  grpc://10.92.92.154:8470
REPLICAS:  8


In [5]:
tf.keras.backend.clear_session()

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
print('Running on TPU ', resolver.cluster_spec().as_dict()['worker'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

Running on TPU  ['10.105.255.34:8470']


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [12]:
(train, val, test), info = tfds.load('fashion_mnist', shuffle_files=True, as_supervised = True, split = ['train[:80%]','train[80%:]', 'test'], with_info = True)

 # EDA

In [13]:
train

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [14]:
print(len(train))
print(len(val))
print(len(test))

48000
12000
10000


In [15]:
x_train, y_train = [], []
for image, label in tfds.as_numpy(train):
    x_train.append(image)
    y_train.append(label)
 
x_val, y_val = [], []
for image, label in tfds.as_numpy(val):
    x_val.append(image)
    y_val.append(label)

UnimplementedError: ignored

In [16]:
tfds.as_dataframe(train.take(5), info)

UnimplementedError: ignored

In [17]:
class_names = info.features["label"].names
n_classes = info.features['label'].num_classes
print(class_names, '\n', n_classes)

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 
 10


In [18]:
np.unique(y_train, return_counts= True)

(array([], dtype=float64), array([], dtype=int64))

In [19]:
np.unique(y_val, return_counts= True)

NameError: ignored

In [20]:
np.unique(x_train)

array([], dtype=float64)

# Fully CNN

In [21]:
train = train.shuffle(1000000).batch(32).prefetch(1)
val = val.shuffle(1000000).batch(32).prefetch(1)
def preprocess_FCNN(image, label):
    #image = tf.image.resize(image, [244,244])
    image = image/255
    return (image, label)

train = train.map(preprocess_FCNN)
val = val.map(preprocess_FCNN)
print(train)

Exception ignored in: <function Executor.__del__ at 0x7fb46b2000d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.UnimplementedError: File system scheme '[local]' not implemented (file: '~/tensorflow_datasets/fashion_mnist/3.0.1/fashion_mnist-train.tfrecord-00000-of-00001')
Exception ignored in: <function Executor.__del__ at 0x7fb46b2000d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.f

<MapDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


Define a function to build the model:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28,28,1]))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(10, 3, activation='relu', padding='valid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)         2

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='Adam')

In [ ]:
model.fit(train, validation_data = val, epochs = 200, callbacks = [tf.keras.callbacks.EarlyStopping(patience=25)])

Epoch 1/200
 210/1500 [===>..........................] - ETA: 2:45 - loss: 2.7845

KeyboardInterrupt: 

# Experiments for later:

In [ ]:
def build_FCNN(hp, input_shape=16):
                #n_hidden = hp.Int("n_hidden", min_value = 1, max_value = 10, step = 1)
                #learning_rate = hp.Float("learning_rate", min_value = 3e-5, max_value = 3e-2)
                model = keras.models.Sequential()
                model.add(keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[28,28,1]))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
                model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
                model.add(keras.layers.Conv2D(256, 3, activation='relu', padding='same'))
                model.add(keras.layers.MaxPooling2D(2))
                model.add(keras.layers.Conv2D(10, , activation='relu', padding='valid'))
                
                
                model.add(keras.layers.Dropout(rate=0.2))
                for layer in range(n_hidden):
                    model.add(keras.layers.Dense(n_neurons, activation="relu"))
                    model.add(keras.layers.Dropout(rate=0.2))
                model.add(keras.layers.Dense(1))
                opt = keras.optimizers.Adam(learning_rate)
                model.compile(loss='mean_squared_error',optimizer=opt)
                return model

Pass build_model to keras_tuner:

In [ ]:
#tuner = keras_tuner.tuners.BayesianOptimization(build_model, objective="val_loss", max_trials = 50, overwrite = True)
tuner = keras_tuner.tuners.RandomSearch(build_model, objective="val_loss", max_trials=200, executions_per_trial=1, overwrite = True)

In [ ]:
tuner.search(train[:,:-2],train[:,-2], epochs = 200,validation_data = (val[:,:-2],val[:,-2]), callbacks=[keras.callbacks.EarlyStopping(patience=25)])

Get parameters and models:

In [ ]:
best_hp = tuner.get_best_hyperparameters(200)
best_models = tuner.get_best_models(200)

In [ ]:
def build_FCNN()